<a href="https://colab.research.google.com/github/gargii21/MLPros/blob/main/XGBOOST_Final50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/stepmix_input50.csv')
df.shape

(50, 397)

In [ ]:
#increamental cha random search

import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

# Prepare your data
X = df.iloc[:, :396].replace(-1, np.nan).values
y = df.iloc[:, 396].map({"Stable": 0, "Slow Decline": 1, "Rapid Decline": 2}).values

# param_grid = {
#     'n_estimators': [10, 20, 50],  # low because we'll train incrementally
#     'max_depth': [3, 5, 7],
#     'learning_rate': [0.01, 0.05, 0.1],
#     'subsample': [0.7, 1.0],
#     'colsample_bytree': [0.7, 1.0],
#     'gamma': [0, 0.1],
#     'min_child_weight': [1, 3],
#     'reg_alpha': [0, 0.1],
#     'reg_lambda': [1, 2]
# }

param_grid = {
    'n_estimators': [50, 100, 200],          # Increase for more robust trees even incrementally
    'max_depth': [3, 5, 7, 9],               # Add deeper trees if data is complex
    'learning_rate': [0.005, 0.01, 0.05, 0.1], # Include slower rates for finer updates
    'subsample': [0.5, 0.7, 0.9, 1.0],       # More variability in sampling data rows
    'colsample_bytree': [0.5, 0.7, 0.9, 1.0],# More variability in sampling features
    'gamma': [0, 0.1, 0.3, 0.5],             # Try stronger pruning options
    'min_child_weight': [1, 3, 5, 7],        # Regularize deeper splits
    'reg_alpha': [0, 0.1, 0.5, 1],           # L1 regularization to avoid overfitting
    'reg_lambda': [1, 1.5, 2, 5]             # L2 regularization
}


model = xgb.XGBClassifier(
    eval_metric='mlogloss',
    use_label_encoder=False
)

random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_grid,
    n_iter=30,
    scoring='accuracy',
    cv=3,
    verbose=1,
    n_jobs=-1,
    random_state=42
)

random_search.fit(X, y)

best_params = random_search.best_params_
print("Best hyperparameters:", best_params)


Fitting 3 folds for each of 30 candidates, totalling 90 fits


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [13:03:32] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Best hyperparameters: {'subsample': 1.0, 'reg_lambda': 1.5, 'reg_alpha': 0.5, 'n_estimators': 100, 'min_child_weight': 1, 'max_depth': 5, 'learning_rate': 0.05, 'gamma': 0.5, 'colsample_bytree': 1.0}


In [ ]:
#increamental model

# # batch_size = 100
# n_samples = X.shape[0]
# n_batches = n_samples // batch_size

# Initialize model with best hyperparameters
incremental_model = xgb.XGBClassifier(
    **best_params,
    eval_metric='mlogloss',
    use_label_encoder=False
)

prev_model = None

for i in range(1,7):
     X_test = np.full((50, 396), np.nan)
     X_test[:, :66*i] = X[:, :66*i]  # <-- take first 66 * i columns from X
     y = df.iloc[:, 396].map({"Stable": 0, "Slow Decline": 1, "Rapid Decline": 2}).values



     incremental_model.fit(
        X_test, y,
        xgb_model=prev_model
     )

     prev_model = incremental_model.get_booster()


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [13:04:06] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [13:04:07] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [13:04:08] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [13:04:08] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [13:04:09] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_e

In [ ]:

def mask_all_after(X, n_unmasked=66):
    X_masked = X.copy()
    X_masked[:, n_unmasked:] = np.nan
    return X_masked

# Masked data: only first 66 features present
for i in range(1,7):
  X_masked = mask_all_after(X, n_unmasked=66*i)

  # Continue training the existing best_model on masked data
  incremental_model.fit(
      X_masked, y,
      xgb_model=incremental_model.get_booster()
  )

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [13:04:20] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [13:04:21] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [13:04:22] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [ ]:
# for j in range(3):
#   for i in range(1, 7):
#     X_masked = mask_all_after(X, n_unmasked=66*i)

#     # Continue training the existing best_model on masked data
#     incremental_model.fit(
#         X_masked, y,
#         xgb_model=incremental_model.get_booster()
#     )

In [ ]:
for j in range(3):
  for i in range(1, 7):
    X_masked = mask_all_after(X, n_unmasked=66*i)

    # Continue training the existing best_model on masked data
    incremental_model.fit(
        X_masked, y,
        xgb_model=incremental_model.get_booster()
    )

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [13:05:03] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [13:05:04] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [13:05:05] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [13:05:06] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [13:05:07] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_e

In [ ]:
from sklearn.metrics import accuracy_score

for i in range(1, 7):
    X_test = np.full((50, 396), np.nan)
    X_test[:, :66*i] = X[:, :66*i]
    preds = incremental_model.predict(X_test)
    acc = accuracy_score(y[:], preds)
    print(f"Accuracy with first {66*i} features: {acc}")

Accuracy with first 66 features: 0.96
Accuracy with first 132 features: 0.98
Accuracy with first 198 features: 0.98
Accuracy with first 264 features: 0.98
Accuracy with first 330 features: 0.98
Accuracy with first 396 features: 0.98


In [ ]:
#TESTING

In [ ]:
dftest = pd.read_csv('/content/drive/MyDrive/Stepmix_test_10.csv')
dftest.shape

(10, 397)

In [ ]:
X = dftest.iloc[:, :396].replace(-1, np.nan).values
y = dftest.iloc[:, 396].map({"Stable": 0, "Slow Decline": 1, "Rapid Decline": 2}).values

from sklearn.metrics import accuracy_score

for i in range(1, 7):
    X_test = np.full((10, 396), np.nan)
    X_test[:, :66*i] = X[:, :66*i]
    preds = incremental_model.predict(X_test)
    acc = accuracy_score(y[:], preds)
    print(f"Accuracy with first {66*i} features: {acc}")

Accuracy with first 66 features: 0.7
Accuracy with first 132 features: 0.7
Accuracy with first 198 features: 0.7
Accuracy with first 264 features: 0.7
Accuracy with first 330 features: 0.7
Accuracy with first 396 features: 0.7
